## Imports

In [1]:
INIT_PATH = "."
DOWNLOAD_DATA = False

# To use google colab processing uncomment following lines

#INIT_PATH = "MyDrive/inference_and_learning"
#from google.colab import drive
#MOUNT_PATH = "/content/drive"
#drive.mount(MOUNT_PATH)
#INIT_PATH = MOUNT_PATH + "/" + INIT_PATH
#DOWNLOAD_DATA = True

# To train set to true, to use pretrained model set to false
RETRAIN = False
LOCAL_TRAIN = True # Only for Local Linear Model
REOPTIMIZE = False

# To avoid out of memory error, adjust 
NUM_THREADS = 1

In [2]:
import os.path,sys
sys.path.append(INIT_PATH)

import networks as nnets
import importlib
importlib.reload(nnets)
import pandas as pd, torch
from typing import Tuple, List
import torch, torchvision

from time import sleep
import json
from IPython.display import display, HTML
torch.set_default_dtype(torch.float16)

In [3]:
DATA_PATH = "." # Path to data folder
if (DOWNLOAD_DATA):
    torchvision.datasets.CIFAR10(root=DATA_PATH + '/data', download=True)
    torchvision.datasets.CIFAR100(root=DATA_PATH + '/data', download=True)
    torchvision.datasets.SVHN(root=DATA_PATH +'/data', download=True)

    torchvision.datasets.CIFAR10(root=DATA_PATH +'/data', download=True,train = False)
    torchvision.datasets.CIFAR100(root=DATA_PATH +'/data', download=True ,train = False) 
    torchvision.datasets.SVHN(root=DATA_PATH +'/data', download=True, split = "test")


## Training
The following code trains network and save result in files

Two steps are realized :
1. Optimization of the hyperparameters from the optim file
2. Training of the network from the train file
    

In [4]:
trainer = nnets.ThreadedTrainer(
    num_threads= NUM_THREADS,
    DATA_PATH=DATA_PATH,
    SAVE_RESULT_PATH= INIT_PATH + "/files/results.json",
    SAVE_NETWORK_PATH=INIT_PATH + "/networks_saved/",
    SAVE_NON_ZERO_PATH = INIT_PATH + "/files/non_zero.json",
    saving_step= 20
)

Trainer ready


### Optimization


You may use this script to test different hyperparameters
Load the network description in the file files/optim.json
The file is a json file with the following structure :
```json
{
    "optim":{ // Hyperparameters to optimize
        "model":{ // Hyperparameters to optimize for the model
            "S-LOCAL":{ // Model you want to optimize hyperparameters on 
                "hidden_size":[64,128,256,512], // List of values to test
                "dropout":[0.1,0.2,0.3,0.4,0.5] // List of values to test
            }
        }
    },
    "networks":{ // List of networks to test
        "S-LOCAL-64-0.1":{ // Name of the network
            //add the network default description here
        }
    },
    "results":{ // Results of the optimization
        "S-LOCAL-64-0.1":{ //Name of the network
            "best":{ 
                // Best hyperparameters found
            },
            "accuracy":0.9 // Accuracy of the best hyperparameters
        }
    }
}
```

In [5]:
if REOPTIMIZE :
    optim = json.load(open(INIT_PATH+"/files/optim.json"))

    list_to_do = [key for key,item in optim["networks"].items() if
                key not in optim["results"] and
                not (not LOCAL_TRAIN and item["model"]["name"] in ["S-LOCAL","D-LOCAL"])]

    for key in list_to_do:
        nnets.network_thread(optim,key, trainer)
    

### Training

You may use this script to train networks from the /files/train.json file

In [5]:
train_net = json.load(open(INIT_PATH+"/files/train.json"))
for key,item in train_net.items():
    if item["model"]["name"] in ["S-LOCAL","D-LOCAL"] and not LOCAL_TRAIN:
        continue
    if not RETRAIN and os.path.isfile(INIT_PATH+"/networks_saved/"+key+".pth"):
        continue
    trainer.add(key,nnets.Network(item),item["model"]["name"] == "S-FC" and key.split("_")[1] == "B-lasso(B=50)")

In [6]:
trainer.progress()

RESULTS :
         Model     |   Optimizer   |    Dataset    ||    optimization_parameters   ||    accuracy   
    ---------------|---------------|---------------||------------------------------||---------------
         S-CONV    |      SGD      |    CIFAR-10   ||              ¤               ||     62.37     
         S-CONV    |      SGD      |   CIFAR-100   ||              ¤               ||     30.46     
         S-CONV    |      SGD      |      SVHN     ||              ¤               ||     85.57     
          3-FC     |      SGD      |    CIFAR-10   ||              ¤               ||     59.44     
          S-FC     |      SGD      |   CIFAR-100   ||              ¤               ||     32.12     
          3-FC     |      SGD      |   CIFAR-100   ||              ¤               ||     30.80     
          S-FC     |      SGD      |      SVHN     ||              ¤               ||     85.64     
          3-FC     |      SGD      |      SVHN     ||              ¤             

### Destroy trainer

In [ ]:
del trainer

## Reproducing the paper


### Table 2

In [15]:
table2 = pd.read_csv(INIT_PATH+"/files_paper/table2.csv")
results = json.load(open(INIT_PATH+"/files/results.json"))

for dataset in ["CIFAR-10","CIFAR-100","SVHN"]:
    for line in range(len(table2)):
        iline = table2.iloc[line].to_dict()
        if iline[dataset] == "nan": continue
        ID = iline["Model"] + "_" + iline["Training_Method"] + "_" + dataset
        if ID in results:
            table2.loc[line,dataset] = "{:.2f}%".format(results[ID])
display(HTML(table2.fillna("").to_html(notebook=True)))

,Model,Training_Method,CIFAR-10,CIFAR-100,SVHN
0,S-CONV,SGD,62.37%,30.46%,85.57%
1,S-LOCAL,SGD,0%,0%,0%
2,MLP (Neyshabur et al. 2019),SGD,58.1%,,84.3%
3,MLP (Mukkamala and Hein 2017),ADAM_RMS,72.2%,39.3%,
4,MLP (Mocanu et al. 2018),SET,74.84%,,
5,MLP (Urban et al. 2017),deep_conv_teacher,74.3%,,
6,MLP (Lin et al. 2016),ZAE,78.62%,,
7,3-FC,SGD,59.44%,30.80%,44.29%
8,S-FC,SGD,59.80%,32.12%,85.64%
9,S-FC,B-lasso(B=0),59.03%,31.11%,87.26%


### figure 3
We can reuse result saved while training table 2

### figure 4